In [1]:
# Import libraries
import pandas as pd
import numpy as np


In [2]:
# Load datasets 
orders = pd.read_csv("olist_orders_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")
items = pd.read_csv("olist_order_items_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")


In [3]:
# Convert date columns to datetime
date_cols = ["order_purchase_timestamp", "order_approved_at", 
             "order_delivered_carrier_date", "order_delivered_customer_date", 
             "order_estimated_delivery_date"]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")


In [4]:
# Drop duplicates (if any)
orders.drop_duplicates(inplace=True)
customers.drop_duplicates(inplace=True)
items.drop_duplicates(inplace=True)
payments.drop_duplicates(inplace=True)
products.drop_duplicates(inplace=True)
reviews.drop_duplicates(inplace=True)


In [5]:
# Merge customers with orders
df = orders.merge(customers, on="customer_id", how="left")


In [6]:
# Merge payments
df = df.merge(payments, on="order_id", how="left")


In [7]:
# Merge items (products inside orders)
df = df.merge(items, on="order_id", how="left")

# Merge products
df = df.merge(products, on="product_id", how="left")

# Merge reviews
df = df.merge(reviews[["order_id","review_score"]], on="order_id", how="left")


In [8]:
# Create month column for trend analysis
df["order_month"] = df["order_purchase_timestamp"].dt.to_period("M")
print(df["order_month"])


0         2017-10
1         2017-10
2         2017-10
3         2018-07
4         2018-08
           ...   
119138    2018-02
119139    2017-08
119140    2018-01
119141    2018-01
119142    2018-03
Name: order_month, Length: 119143, dtype: period[M]


In [9]:
# Delivery performance (days)
df["delivery_days"] = (df["order_delivered_customer_date"] - df["order_approved_at"]).dt.days
print(df["delivery_days"])

0          8.0
1          8.0
2          8.0
3         12.0
4          9.0
          ... 
119138    22.0
119139    24.0
119140    17.0
119141    17.0
119142     7.0
Name: delivery_days, Length: 119143, dtype: float64


In [10]:
# Average order value
df["avg_order_value"] = df.groupby("order_id")["payment_value"].transform("mean")
print(df["avg_order_value"])

0          12.903333
1          12.903333
2          12.903333
3         141.460000
4         179.120000
             ...    
119138    195.000000
119139    271.010000
119140    441.160000
119141    441.160000
119142     86.860000
Name: avg_order_value, Length: 119143, dtype: float64


In [13]:
#handle missing value
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
df["review_score"] = df["review_score"].fillna(df["review_score"].mode()[0])


In [14]:
# Save Cleaned Dataset
df.to_csv("cleaned_ecommerce_dataset.csv", index=False)

print(" Cleaned dataset saved as cleaned_ecommerce_dataset.csv")
print("Shape of dataset:", df.shape)


 Cleaned dataset saved as cleaned_ecommerce_dataset.csv
Shape of dataset: (119143, 34)


SUMMARY


In [15]:
# What I Have Done (Data Cleaning and Preprocessing)

# 1. Load Dataset
# 2. Format Date Columns
# 3. Remove Duplicates
# 4. Merge Tables
# 5. Feature Engineering (New Columns)
# 6. Handle Missing Values
# 7. Export Clean Dataset
